In [46]:
import pandas as pd

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA


import tkinter as tk
from tkinter import scrolledtext
from langchain_ollama import OllamaLLM
import os

In [ ]:
# Charger les fichiers CSV
data_1 = pd.read_csv('/home/cytech/test/Rag_covid/data/data_preprocessing/Age_Preprocessing.csv')
data_2 = pd.read_csv('/home/cytech/test/Rag_covid/data/data_preprocessing/Diabetes_Preprocessing.csv')
data_3 = pd.read_csv('/home/cytech/test/Rag_covid/data/data_preprocessing/Overweight_or_obese_Preprocessing.csv')


In [54]:

# Fusionner les données
combined_data = pd.concat([data_1, data_2, data_3], ignore_index=True)

In [83]:
# Fonction pour segmenter un document en morceaux de 512 tokens
def segmenter_texte(texte, longueur_max=512):
    tokens = texte.split()
    segments = []
    for i in range(0, len(tokens), longueur_max):
        segment = ' '.join(tokens[i:i + longueur_max])
        segments.append(segment)
    return segments

In [84]:
# Appliquer la segmentation à chaque document de la colonne 'context'
documents = combined_data['context'].dropna().tolist()
segmented_docs = []
for doc in documents:
    segmented_docs.extend(segmenter_texte(doc))

In [85]:
# Initialiser le modèle d'embedding avec une dimension réduite
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/static-retrieval-mrl-en-v1",
    model_kwargs={'device': 'cpu', 'truncate_dim': 1024}
)

In [86]:
# Créer l’index Chroma dans un nouveau dossier
persist_dir = "./chroma_static_mrl"
vectorstore = Chroma.from_texts(segmented_docs, embedding_model, persist_directory=persist_dir)


In [87]:
# Créer le système de récupération
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


In [88]:
# Initialiser le modèle LLaMA
llm = Ollama(model="llama3.2:3b")


In [89]:
# Construire la chaîne QA
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [91]:
docs = retriever.get_relevant_documents(query)
print("📝 Docs récupérés :")
for d in docs:
    print(d.page_content[:300])  # Affiche les premiers caractères


📝 Docs récupérés :
Date: 2020-05-16. Study: Severe obesity is associated with higher in-hospital mortality in a cohort of patients with COVID-19 in the Bronx, New York. Study Link: https://www.ncbi.nlm.nih.gov/pubmed/32422233/. Journal: Metabolism. Severe: AOR=1.26. Severe Lower Bound: 0.58. Severe Upper Bound: 2.73. 
Date: 2020-05-16. Study: Severe obesity is associated with higher in-hospital mortality in a cohort of patients with COVID-19 in the Bronx, New York. Study Link: https://www.ncbi.nlm.nih.gov/pubmed/32422233/. Journal: Metabolism. Severe: AOR=1.26. Severe Lower Bound: 0.58. Severe Upper Bound: 2.73. 


/tmp/ipykernel_4808/2251331332.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [92]:
# Exemple de requête
query = "Give me figures on covid and overweight"
response = qa_chain.run(query)
print(response)


KeyboardInterrupt: 

In [15]:
query = "donne moi des informations sur le diabète et le covid"
response = qa_chain.run(query)
print(response)


/tmp/ipykernel_600982/72631302.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


D'après les études présentées, il est évident que le diabète augmente le risque de sévérité et de mortalité liés au COVID-19. Voici quelques points clés tirés des données :

1. **Sévérité du COVID-19 chez les personnes diabétiques** : Les études suggèrent que les personnes diabétiques ont un risque plus élevé de développer une forme sévère de la maladie, avec un facteur de risque (OR) compris entre 1,02 et 2,75. Cela signifie que les personnes diabétiques sont plus susceptibles d'être hospitalisées ou d'avoir besoin d'un suivi médical intense en cas de COVID-19.

2. **Mortalité liée au COVID-19 chez les personnes diabétiques** : Les études indiquent également que le risque de mortalité est plus élevé chez les personnes diabétiques, avec un facteur de risque (OR) compris entre 1,07 et 2,80. Cela signifie que les personnes diabétiques ont un risque plus important d'être décédées en raison du COVID-19 comparé à la population générale.

3. **Facteurs de risque associés** : Certaines études

In [23]:
query = "Give me figures on covid and overweight"
response = qa_chain.run(query)
print(response)


Based on the provided studies, here are some figures related to COVID-19 and obesity:

1. Obesity as a risk factor for COVID-19 mortality:
	* One study found an increased risk of death from COVID-19 in obese patients (Fatality_number: 0.26).
	* Another study found a similar association with an OR of 2.06, although the p-value was not significant.
	* A third study found that obesity was associated with an increased risk of death from COVID-19 (AOR 1.4), but this was only for patients older than 50.
2. Mortality rates:
	* The first study found a fatality rate of 0.26% among obese patients.
	* The second study found a fatality rate of 1.11% among obese patients, although the p-value was not significant.
	* The third study did not provide a specific mortality rate for obese patients.

It's worth noting that the results are inconsistent across studies, and more research is needed to fully understand the relationship between obesity and COVID-19 mortality.
